# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk
import numpy as np
from joblib import dump
from pandas import DataFrame, Series, read_sql_table
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = read_sql_table('disaster_messages', engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis=1)
category_names = Y.columns

In [3]:
df.related.unique()

array([1, 0], dtype=int64)

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text: str) -> list:
    """
    INPUT:
    text - string containing the message
    
    OUTPUT:
    tokens - List of tokens

    This function tokenizes the input text by using URL replacement, normalization, punctuation removal, tokenization,
    lemmatization and stemming
    """
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'   
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        # I do not lowercase words as it is done in the CountVectorizer() object before using that function by default 
        # (lowercase=True)
        clean_tok = lemmatizer.lemmatize(tok).strip()  
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42, n_estimators=5)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
# Splitting the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x000001F34C3D63E0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5,
                                                                        random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
print(classification_report(Y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.81      0.93      0.86      3959
               request       0.71      0.44      0.54       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.66      0.59      0.62      2156
          medical_help       0.54      0.15      0.24       431
      medical_products       0.53      0.14      0.23       264
     search_and_rescue       0.50      0.08      0.14       151
              security       0.25      0.01      0.02       106
              military       0.62      0.18      0.28       175
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.26      0.39       344
                  food       0.74      0.56      0.64       586
               shelter       0.71      0.31      0.43       487
              clothing       0.47      0.10      0.17        79
                 money       0.33      

C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
print(pipeline.get_params().keys())

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__monotonic_cst', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__esti

In [21]:
# specify parameters for grid search
parameters = {
   'vect__ngram_range': ((1, 1), (1, 2)),
   'clf__estimator__n_estimators': [5, 10, 20],
   'clf__estimator__min_samples_leaf': [2, 3, 4]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=3, cv=2)

# fit the grid search 
cv.fit(X_train, Y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(token_pattern=None,
                                                        tokenizer=<function tokenize at 0x000001F34C3D63E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5,
                                                                                               random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__min_samples_leaf': [2, 3, 4],
                         'clf__estimator__n_estimators': [5, 10, 20],
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
y_pred = pipeline.predict(X_test)
print("Best Parameters:", cv.best_params_)
print(classification_report(Y_test, y_pred, target_names=category_names))

Best Parameters: {'clf__estimator__min_samples_leaf': 2, 'clf__estimator__n_estimators': 20, 'vect__ngram_range': (1, 2)}
                        precision    recall  f1-score   support

               related       0.81      0.93      0.86      3959
               request       0.71      0.44      0.54       902
                 offer       0.00      0.00      0.00        25
           aid_related       0.66      0.59      0.62      2156
          medical_help       0.54      0.15      0.24       431
      medical_products       0.53      0.14      0.23       264
     search_and_rescue       0.50      0.08      0.14       151
              security       0.25      0.01      0.02       106
              military       0.62      0.18      0.28       175
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.26      0.39       344
                  food       0.74      0.56      0.64       586
               shelter       0.71      0.31  

C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\flopezr81\anaconda3\

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [6]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return 1
        return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = Series(X).apply(self.starting_verb)
        return DataFrame(X_tagged)

In [7]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)), 
                      ('tfidf', TfidfTransformer())])),
                        ('starting_verb', StartingVerbExtractor())      
                              ])),
     ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42, n_estimators=5)))])


In [11]:
# specify parameters for grid search
parameters2 = {
   'features__text_pipeline__vect__ngram_range': [(1, 2)],
   'clf__estimator__n_estimators': [5],
   'clf__estimator__min_samples_leaf': [2]
}

# create grid search object
cv2 = GridSearchCV(pipeline2, param_grid=parameters2, n_jobs=-1, verbose=3, cv=2)

# fit the grid search 
cv2.fit(X_train, Y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(token_pattern=None,
                                                                                                         tokenizer=<function tokenize at 0x000002834356B2E0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=5,
                                                                                               random_state=42)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__min_samples_leaf': [2],
                         'clf__estimator__n_estimators': [5],
                         'features__text_pipeline__vect__ngram_range': [(1,
                                                                         2)]},
             verbose=3)

In [12]:
y_pred2 = pipeline2.predict(X_test)
print("Best Parameters:", cv2.best_params_)
print(classification_report(Y_test, y_pred2, target_names=category_names))

NotFittedError: Vocabulary not fitted or provided

### 9. Export your model as a pickle file

In [24]:
dump(cv, "model.pkl")

['model.pkl']

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.